# Jess_cfos_for_gallery-webdriver
I used this notebook to make the lightserv gallery gif of Thalamic expression in Jess' c-Fos an21 subject.

This notebook saves a bunch of pngs which I then convert to gif using a python script, e.g.:
```python
from PIL import Image
import glob, os
 
ss_dir = '/home/ahoag/ngdemo/screenshots/cfos_an21'
savename = os.path.join(ss_dir,'cfos_an21.gif')
if __name__ == '__main__':
	# Create the frames
	pngs = sorted(glob.glob(ss_dir+'/*png'))
	print(pngs)
	frames = [Image.open(png) for png in pngs]
	
	# Save into a GIF file that loops forever
	frames[0].save(savename, format='GIF', append_images=frames[1:], save_all=True, duration=200, loop=0)
```
I was having trouble getting the segment properties to be displayed in the png screenshots - I opened an issue on the google github for this: https://github.com/google/neuroglancer/issues/233. It can be achieved using the selenium webdriver as done below. In order to get the webdriver to work I had to pip install selenium (not included with neuroglancer pip), then I had to download the chromedriver binary from here: https://chromedriver.chromium.org/downloads since I was using google chrome as my browser. Make sure to put the "chromedriver" executable in your shell path. 

In [6]:
import neuroglancer as ng
from neuroglancer import webdriver
import os
import matplotlib.pyplot as plt
%matplotlib inline

## Set the neuroglancer client to be one that is hosted locally


In [2]:
# Use localhost for static files
ng.set_static_content_source(url='http://localhost:8080')

In [3]:
# Start a web viewer and load in my three layers, which are hosted on my local machine.
# Note that I used raw-space annotation layer for the cells, but I put a high "limit" on it (~500000)
# So that the number of cells displayed was low -- with the native sampling the cells are too dense
# to see anything else
viewer = ng.Viewer()
with viewer.txn() as s:
    s.layers['rawdata_an21'] = ng.ImageLayer(source='precomputed://http://localhost:1337'
    )
    s.layers['rawatlas_an21'] = ng.SegmentationLayer(source='precomputed://http://localhost:1338'
    )
    s.layers['rawcells_an21'] = ng.AnnotationLayer(source='precomputed://http://localhost:8086'
    )
print(viewer)

http://127.0.0.1:34237/v/cef0fb86dee23295ebcc2f897b6dd05ff9bad33f/


In [4]:
viewer = ng.Viewer()
with viewer.txn() as s:
    s.layers['rawatlas_an21'] = ng.SegmentationLayer(source='precomputed://http://localhost:1338'
    )
print(viewer)

http://127.0.0.1:34237/v/cda13063be101eec422dac8a73814375e01fddd6/


In [5]:
# Start the webdriver which should open a new window
webdriver = webdriver.Webdriver(viewer, headless=False,)

AttributeError: module 'neuroglancer' has no attribute 'webdriver'

In [ ]:
webdriver.driver.set_window_size(1700,800)

In [ ]:
# Here is where I take the sequence of screenshots 

# Set up the initial frame
with webdriver.viewer.txn() as s:
    s.layout = 'yz-3d'
    s.position = [1710,1276,331]
    
    s.cross_section_orientation = [0.7071067690849304, 0, 0, 0.7071067690849304]
    s.cross_section_scale = 5.5
    s.projection_scale = 3900
    s.projection_orientation = [0.65080833, 0.17286249, 0.20033664, 0.7116406 ]
    seglayer = s.layers['rawatlas_an21']
    seglayer.segments = [549, 262, 741, 149, 629, 599, 1113, 733]
    seglayer.segment_query = "549, 262, 741, 149, 629, 599, 1113, 733"
    annotlayer = s.layers['rawcells_an21']
    annotlayer.visible=False
    annotlayer.shader = "\nvoid main() {\n  setColor(vec4(defaultColor(), 0.5));\n}"
    s.selected_layer.layer = 'rawatlas_an21' 
    s.selected_layer.visible = True
    s.selected_layer.size = 715
    s.layers['rawatlas_an21'].tab = "segments"
    s.show_axis_lines=False

savedst = '/home/ahoag/ngdemo/screenshots/cfos_an21'
# Do a sweep through sagittal sections
ss_ii = 0
for i in range(400,1600,50):
    with webdriver.viewer.txn() as s:
        s.position = [i,1265,359] #the xy coords here are from the neuroglancer window
    webdriver.driver.save_screenshot(os.path.join(savedst,f'{ss_ii}.png'.zfill(6)))
    ss_ii += 1
# Turn on cell layer and sweep back
with webdriver.viewer.txn() as s:
    annotlayer = s.layers['rawcells_an21']
    annotlayer.visible=True
for i in range(1550,350,-50):
    with webdriver.viewer.txn() as s:
        s.position = [i,1265,359] #the xy coords here are from the neuroglancer window
    webdriver.driver.save_screenshot(os.path.join(savedst,f'{ss_ii}.png'.zfill(6)))
    ss_ii += 1